# Lab 7: Human motion generation

## Advanced deep learning

## Setup

### Setup dataset


Please access this Google Drive folder: [link](https://drive.google.com/drive/folders/1V5yzlwBPSNVPj33SfDHnvMykXISh3CyB?usp=sharing) and create a shortcut in the root of your Google Drive `/content/drive/MyDrive/`.

In [1]:
from google.colab import drive
drive.mount('/content/drive')
!ls /content/drive/MyDrive/humanml3d-data

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
caption_clip  caption_raw  checkpoints	humanml3d_test_split.txt  smplh  smpl_rifke


### Setup environment

Make sure you're running on a T4 GPU Colab instance; if not, activate it.

In [2]:
!nvidia-smi

Mon Mar  3 21:55:03 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
import torch
torch.cuda.is_available()

True

Clone the lab repository.

/!\ I forked the repository on my own github so that I can more easily modify it and keep a version (and give the link to the forked repo <a href="https://github.com/milletyann/lab7-CSC52087EP/tree/main">here</a> for the code correction) /!\

My results are in the /results folder of the repo.

In [4]:
!git clone https://github.com/milletyann/lab7-CSC52087EP.git
#!git clone https://github.com/robincourant/lab-MotionDiT.git

fatal: destination path 'lab7-CSC52087EP' already exists and is not an empty directory.


In [5]:
%cd lab7-CSC52087EP
#%cd lab-MotionDiT
!ln -s /content/drive/MyDrive/humanml3d-data ./ # Plug the dataset in the repo

/content/lab7-CSC52087EP
ln: failed to create symbolic link './humanml3d-data': File exists


Install required libraries

In [6]:
!pip install hydra-core
!pip install pyrender
!pip install smplx
!pip install torchtyping
!pip install lightning
!pip install ema_pytorch

  Using cached smplx-0.1.28-py3-none-any.whl.metadata (10 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-manylinu

## Human motion dataset and representation

### HumanML3D dataset

In [16]:
# use to see where we are
!pwd
!ls

/content/lab7-CSC52087EP
configs  humanml3d-data  lab7_CSC52087EP.ipynb	lab7_CSC52087EP.pdf  README.md	src  utils


In [17]:
# use to navigate in the dirs
%cd lab7-CSC52087EP

[Errno 2] No such file or directory: 'lab7-CSC52087EP'
/content/lab7-CSC52087EP


In [42]:
import os
import numpy as np

motions_path = 'humanml3d-data/smpl_rifke'
descr_path = 'humanml3d-data/caption_raw'

nb_motions = 0
nb_descr = 0
motion_length = 0
descr_length = 0

motion_files = [f for f in os.listdir(motions_path) if f.endswith(".npy")]

for motion in motion_files:
  motion_data = np.load(os.path.join(motions_path, motion))
  nb_motions += 1
  motion_length += motion_data.shape[0]

for motion in motion_files:
  text_file = os.path.splitext(motion)[0] + ".txt"

  if os.path.isfile(os.path.join(descr_path, text_file)):
    with open(os.path.join(descr_path, text_file), 'r') as f:
      descriptions = f.readlines()
      nb_descr += len(descriptions)
      descr_length += sum(len(desc.strip()) for desc in descriptions)

avg_motion_length = motion_length / nb_motions
avg_descr_length = descr_length / nb_descr

print(f"Number of motions: {nb_motions}")
print(f"Number of descriptions: {nb_descr}")
print(f"Average motion length: {avg_motion_length}")
print(f"Average description length: {avg_descr_length}")

/
Number of motions: 3893
Number of descriptions: 11645
Average motion length: 143.92011302337528
Average description length: 66.92219836839845


#### Question 1: How many motions and descriptions are included in the HumanML3D dataset, and what are their average lengths?

According to <a href="https://paperswithcode.com/dataset/humanml3d#:~:text=Motions%20From%20Text-,HumanML3D%20is%20a%203D%20human%20motion%2Dlanguage%20dataset%20that%20originates,(e.g.%2C%20'dancing').">this page</a> from PapersWithCode, the dataset contains: 14616 motions, 44970 descriptions. The average motion length is 7.1 seconds, and average description length is 12 words.

The previous code is an attempt to verify that, but apparently the values are differents. It took a really long time to run and I have to continue the lab so maybe I will try to fix it later.<br>

My results:<br>
Number of motions: 3893 <br>
Number of descriptions: 11645 <br>
Average motion length: 143.92011302337528 <br>
Average description length: 66.92219836839845 <br>

### SMPL representation


#### Question 2: What are the different input parameters of the SMPL model used to infer the vertices, and what do they represent? Hint: Check the official code of SMPL models.

According to <a href="https://github.com/vchoutas/smplx/blob/main/examples/demo.py">this file</a> in this github repository, the input parameters of a SMPL model are:<br>
- betas: the overall body shape params (for instance height, weight,proportions, ...)<br>
- body_pose: angles of body joints<br>
- global_orient: global orientation of the body in the 3D space. I believe it's just the rotation of the overall body but I couldn't figure out at 100% confidence.<br>
- transl: global translation of the body in the 3D space.<br>
- gender: which gender is being considered (male, female, neutral).<br>
- age.<br>
- kid_template_path: path to locate the kid template. This part is not commented, but I think it is triggered when the age parameter is below a certain value, or maybe even fixed to an 'infant' special value or something like that.<br>

There are more parameters but I've put the ones which made the most sense to me.

#### Code 2
*Complete `visualize_smpl.py`*

In [18]:
!HYDRA_FULL_ERROR=1 PYTHONPATH=$(pwd) python src/visualize_smpl.py

import moviepy.editor
moviepy.editor.ipython_display("./smpl.mp4")

[2025-03-03 21:59:47,354][numexpr.utils][INFO] - NumExpr defaulting to 2 threads.
/content/lab7-CSC52087EP/src/datasets/modalities/human_dataset.py:44: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this

  if event.key is 'enter':



## Model architectures

### Config A: Incontext

#### Code 3
*Complete `src/models/modules/incontext.py`*

In [19]:
!PYTHONPATH=$(pwd) python src/models/modules/incontext.py

Test passed!


### Config B: AdaLN

#### Question 3: Given γ and β the shift and scale parameters, write the AdaLN operation:

adaLN(x,γ,β) = γ * ((x - mean(x)) / std(x)) + β

<br>
(x - mean(x)) / std(x) is the normalization part.<br>
γ * (...) + β is the scaling and shifting part.

#### Code 4
*Complete `src/models/modules/adaln.py`*

In [22]:
!PYTHONPATH=$(pwd) python src/models/modules/adaln.py

torch.Size([2, 5, 16])
torch.Size([2, 16])
torch.Size([2, 16])
torch.Size([2, 5, 16])
torch.Size([2, 16])
torch.Size([2, 16])
torch.Size([2, 5, 16])
torch.Size([2, 16])
torch.Size([2, 16])
torch.Size([2, 5, 16])
torch.Size([2, 16])
torch.Size([2, 16])
torch.Size([2, 5, 16])
torch.Size([2, 16])
torch.Size([2, 16])
Test passed!


### Config C: Cross attention

#### Question 4: Given WQ, WK , WV ,as the query, key, and value weights, respectively, and d the token dimension, write the cross-attention operation between the input x and the condition c:

CA(x,c) = (c * WV) * softmax((x * WQ) * (c * WK)^T / sqrt(d))

<br>
- x*WQ : projects the input x using the queries matrix.<br>
- c*WK : projects the conditioning tensor c using the keys matrix.<br>
- c*WV : projects the conditioning tensor c using the values matrix.<br>
- the softmax and what is in it is the classical cross-attention equation.<br>
- softmax*(c*WK) : multiplication of the attention weights with the projected values, we obtain the weigthed representation of the conditioning iniformation.

#### Code 5
*Complete `src/models/modules/cross attention.py`*

In [37]:
!PYTHONPATH=$(pwd) python src/models/modules/cross_attention.py

Test passed!


#### Question 5: What is the advantage of using AdaLN for conditioning, particularly in comparison to the cross-attention approach?

- The computational complexity of AdaLN is lower than the cross-attention. Cross-attention computes to attention weights for every pair (input, conditioning) tokens. So the complexity is quadratic in the sequence length, but on the other side AdaLN has a linear complexity in the in the features channels.<br>
- Due to that, the inference is faster. It's nice for real-time ifnerence.<br>
- It uses less memory, simpler to implement (in my opinion, maybe it is different for other people).

## Diffusion framework

### DDPM

#### Code 6
*Complete `src/training/losses/ddpm.py`*

In [42]:
!PYTHONPATH=$(pwd) python src/training/losses/ddpm.py

Test passed!


#### Code 7
*Complete `src/training/sampler/ddpm.py`*

In [43]:
!PYTHONPATH=$(pwd) python src/generate.py batch_size=1 diffuser/sampler@diffuser.test_sampler=ddpm seed=2

import moviepy.editor
moviepy.editor.ipython_display("./generation_ddpm_incontext.mp4")

Seed set to 2
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(limit_predict_batches=1)` was configured so 1 batch will be used.
/content/lab7-CSC52087EP/src/metrics/modules/tmr.py:132: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where

### DDIM

#### Question 6: What are the advantages of DDIM over DDPM sampling, and what is the key difference between them?

One big advantage of DDIM over DDPM is the sampling. In DDIM it is faster, it makes larger steps in the reverse diffusion process (so creating an image after training is faster) in skipping some iterations (the sampling process is deterministic). DDPM uses Markov chains to progressively "noisify" an image, and it can take a lot of steps.<br>
The fact that the DDIM is deterministic also helps with diversity and quality of the samples.<br><br>

The key difference between DDPM and DDIM are:
- DDPM: follows Markov chain, so each step of the reverse diffusion process is based on the previous one. So this is a stochastic sampling.<br>
- DDIM: follows a non-Markovian process, each step can depend on several previous other. It a deterministic sampling because it doesn't take the noise term like DDPM. This is what allows it to make large steps and be more efficient in the generation of a sample.<br>


references: lecture7, [<a href="https://medium.com/better-programming/diffusion-models-ddpms-ddims-and-classifier-free-guidance-e07b297b2869#092e">1</a>], [<a href="https://sachinruk.github.io/blog/2024-02-11-DDPM-to-DDIM.html">2</a>]


#### Code 8
*Complete `src/training/sampler/ddim.py`*

In [50]:
!PYTHONPATH=$(pwd) python src/generate.py batch_size=1 diffuser/sampler@diffuser.test_sampler=ddim seed=2

import moviepy.editor
moviepy.editor.ipython_display("./generation_ddim_incontext.mp4")

Seed set to 2
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(limit_predict_batches=1)` was configured so 1 batch will be used.
/content/lab7-CSC52087EP/src/metrics/modules/tmr.py:132: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where

#### Question 7: Are there any qualitative differences between the generated samples from the sampling methods? If yes, what are they?

The difference between DDPM and DDIM is clearly identifiable here. While DDPM creates a human pose with a contorted body, with no real movement and moving really fast, DDIM has a real human stature, moves normally, eventhough the movement are not convincing for a normal, it could be already convincing for a drunk one.

## Result analysis

### Qualitative analysis

#### Code 9

In [46]:
!PYTHONPATH=$(pwd) python src/generate.py batch_size=1 diffuser/network=incontext \
checkpoint_path=./humanml3d-data/checkpoints/incontext.ckpt

import moviepy.editor
moviepy.editor.ipython_display("./generation_ddpm_incontext.mp4")

Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(limit_predict_batches=1)` was configured so 1 batch will be used.
/content/lab7-CSC52087EP/src/metrics/modules/tmr.py:132: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case wher

In [47]:
!PYTHONPATH=$(pwd) python src/generate.py batch_size=1 diffuser/network=adaln \
checkpoint_path=./humanml3d-data/checkpoints/adaln.ckpt

import moviepy.editor
moviepy.editor.ipython_display("./generation_ddpm_adaln.mp4")

Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(limit_predict_batches=1)` was configured so 1 batch will be used.
/content/lab7-CSC52087EP/src/metrics/modules/tmr.py:132: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case wher

In [48]:
!PYTHONPATH=$(pwd) python src/generate.py batch_size=1 diffuser/network=cross_attention \
checkpoint_path=./humanml3d-data/checkpoints/cross_attention.ckpt

import moviepy.editor
moviepy.editor.ipython_display("./generation_ddpm_cross_attention.mp4")

Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(limit_predict_batches=1)` was configured so 1 batch will be used.
/content/lab7-CSC52087EP/src/metrics/modules/tmr.py:132: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case wher

#### Question 8: Are there any qualitative differences between the generated samples from the different architectures? If yes, what are they?

I don't notice any qualitative differences between the 3 architectures results. The 3 results are really bad in my opinion, the body is moving really fast and there is no true human pose, the body is contorted on each video.<br>
I don't know if the reason is because a lab work is not enough to get good results or if I did something wrong in my code, but I'm leaning towards the second option. However, I tried differents things in the code but I couldn't manage to get better results whatsoever.<br>
We compared with the DDIM earlier, and we saw the results were good for DDIM, so I suppose my implementation errors land in the DDPM part and not in the architectural ones (incontext, cross-attention, ...).<br>

### Quantitative analysis

#### Question 9: What is the main assumption about the reference and generated feature spaces when computing the Fréchet Distance?

Assumption: reference and generated feature spaces are following a multivariate Gaussian distribution (so features are supposedly normally distributed according to some mean and variance).<br>
- reference feature space: feature space of the real data, the model has to try to mimic this.<br>
- generated feature space: feature space of the data generated by the model.<br>

#### Code 10
*Complete src/metrics/frechet.py*

#### Code 11
*Complete src/metrics/similarity.py*

#### Code 12

#### Bonus 1: Review the code in src/metrics/motion text.py and src/metrics/prdc.py. Then, explain the meaning of the following metrics: R1, R2, R3, and PRDC.

R1/2/3 are recalls metrics, the idea is to measure how good the generated samples can be retrieved in a base of real data, based on some similarity metric.
- R1: Recall at 1, the percentage of generated samples that are the closest to their corresponding real samples (in the feature space).
- R2: Recall at 2, same thing but for the top 2 closest samples
- R3: Recall at 3, same thing but for the top 3 closest sample.
For these 3 metrics, the highest the better (better quality samples, more similar to real samples).
- PRDC: The acronym is Precision Recall Curve Distance (now I wonder why isn't it called PRCD). It's the combainaison of recall and precision. The PR curve is the recall in the x-axis and the precision in the y-axis. Each point of the curve is for a fixed similarity threshold. So PRDC is the distance between the PR curve of generated samples, and the PR curve of real data, so here the lower the better.

In [54]:
!PYTHONPATH=$(pwd) python src/evaluate.py diffuser/network=incontext \
checkpoint_path=./humanml3d-data/checkpoints/incontext.ckpt

Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(limit_predict_batches=1)` was configured so 1 batch will be used.
[2025-03-04 00:35:02,141][OpenGL.acceleratesupport][INFO] - No OpenGL_accelerate module loaded: No module named 'OpenGL_accelerate'
/content/lab7-CSC52087EP/src/metrics/modules/tmr.py:132: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlis

In [55]:
!PYTHONPATH=$(pwd) python src/evaluate.py diffuser/network=adaln \
checkpoint_path=./humanml3d-data/checkpoints/adaln.ckpt

Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(limit_predict_batches=1)` was configured so 1 batch will be used.
[2025-03-04 00:43:39,517][OpenGL.acceleratesupport][INFO] - No OpenGL_accelerate module loaded: No module named 'OpenGL_accelerate'
/content/lab7-CSC52087EP/src/metrics/modules/tmr.py:132: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlis

In [56]:
!PYTHONPATH=$(pwd) python src/evaluate.py diffuser/network=cross_attention \
checkpoint_path=./humanml3d-data/checkpoints/cross_attention.ckpt

Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(limit_predict_batches=1)` was configured so 1 batch will be used.
[2025-03-04 00:54:35,305][OpenGL.acceleratesupport][INFO] - No OpenGL_accelerate module loaded: No module named 'OpenGL_accelerate'
/content/lab7-CSC52087EP/src/metrics/modules/tmr.py:132: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlis

#### Question 10: Given the computed scores, what can you say?

Looking at the 3 .csv files, we can notice several things:<br>
- for each file, its 640 rows are extremely similar. Even the fid, which I suppose is the identifier of each sample is the same. This leads me to question the code I added because it seems a bit strange unless I missed something in my understanding.
- The tmr_score represent the similarity between the generated motions and the text description. It is nearly 0 for each sample of each architecture, which corroborates the visual inspection we made earlier with the nonsensical movements.
- The precision is relatively high in all 3 architectures (0.78, 0.72 and 0.67 respectively for incontext, adaln and cross attention)
- The recall is nearly 0 for every sample of every architecture, this shows that the retrieved motions are not very relevant and linked to the text description.
- The density represent the diversity of the space of generated motions. All architectures have a moderate density (between 0.43 and 0.61), so the generations are a bit diverse but still quite clustered together.
- The coverage measures how much the target data is represented by the generated motions. Here it is very low for all architectures (between 0.18 and 0.21), so the generated samples represent only a quite small portion of the target data.
- R1/R2/R3 are the recall at 1/2/3 are the percentages of generation that are the closest to their real sample. All values are very low (nearly 0) for every sample with every model, so this finishes to show that the generated motions are nowhere near similar to real motions.

#### Bonus 2: Here we have evaluated our models on 10×64 samples. How reliable do you think each metric is, particularly FDTMR, and why? What should we do?

- Evaluating on 640 (10x64) samples is probably not sufficient to get a good idea of the diversity of the data distribution, and even more when the data is very complex like this.
- Since the number of samples is low, maybe it is not representative of the real data distribution, therefore making the metrics not really able to give a meaningul insight.
- Specifically to FDTMR, this metric assume that the features extracted (from text and motion) follow a multi-variate Guassian distribution. For our small sample it is perhaps not true anymore, and so the reliability of the metric is affected.


Solutions:
- Increase the number of sample to evaluate.
- Do cross-validation? On different subsets of the data, in order to check if the non-representability of our first subset is truly not representative or if it is another problem.
- Change the metrics.
- Inspect the results qualitatively (looking at the videos), this is a bit what we already did but we can maybe push this further.